In [312]:
# Essential Imports
import zipfile
import pandas as pd
import seaborn as sns
import sklearn as skt
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations
from collections import defaultdict
from datetime import datetime
import pickle
import warnings
warnings.simplefilter("ignore")
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder


In [313]:
# Read the data into the panda dataFrames
data = pd.read_csv('/Users/ashvtol/Documents/dsfProjectData/fullData.csv',parse_dates=["Date"])

In [314]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [114]:
check=pd.DataFrame(data['Item Description'].value_counts())

KeyboardInterrupt: 

In [ ]:
check.head(200)

In [214]:
basket = (data[(data['Store Name']=='11730 BETHPAGE') & (data['Line Item Transaction Type']=='Sale')])

In [215]:
basket.shape

(1458272, 40)

In [216]:
basket_clean = basket[['Receipt Number', 'Item Description', 'Net Sales Units']]

In [217]:
basket_clean

,Receipt Number,Item Description,Net Sales Units
0,G13674,FASTENERS,1
24,G13679,GRN PAINT THINNER GL,1
25,G13679,PAINT QT SATN BLK OUTDOR,1
26,G13680,JNT CMPND ALL PURP 4.5G,1
33,G13681,KEROSENE 1-K WC 1G MTL,1
...,...,...,...
32955445,F30938,CMN Donations,1
32955453,F30939,"CM EXT BAR 20"" 3/8""DR",1
32955454,F30939,"CM SLTD SCRWDR 5/16""X6""",1
32955455,F30939,"ACE FAUCET LOCKNUT1/2""IP",1


In [218]:
basket_clean.isna().sum()


Receipt Number      0
Item Description    0
Net Sales Units     0
dtype: int64

### Removing blanks and coupons/donations from item description

In [219]:
basket_clean['Item Description'].value_counts()

FASTENERS/SCREWS/NAILS/BOLTS ETC    48584
CMN Donations                       44708
FASTENERS                           42716
KEY KWIKSET KW1-ACE250PK             9465
BIRDSEED WILDBIRD 20#ACE             8981
                                    ...  
BRUSH CUP WIRE1/8"DREMEL                1
1 1/2" FLOWER KNOB-ANT.SLVR             1
SPRAYR SHUT OFF METAL                   1
DISTANCE MEASURE LSR100'                1
DRILL 1/2"MAKITA 6302H                  1
Name: Item Description, Length: 42901, dtype: int64

In [220]:
basket_clean = basket_clean[basket_clean['Item Description']!= 'CMN Donations']
basket_clean = basket_clean[basket_clean['Item Description']!= '50% OFF 1 ITEM UNDER $30']
basket_clean = basket_clean[basket_clean['Item Description']!= 'THANKSGIVING SALE 15%OFF']
basket_clean.loc[(basket_clean['Item Description'] == 'FASTENERS'), 'Item Description'] = 'FASTENERS/SCREWS/NAILS/BOLTS ETC'

In [221]:
basket_clean.shape

(1412617, 3)

### Remove items purchased less than threshold limit

In [222]:
Above2k = basket_clean['Item Description'].value_counts()

In [223]:
Above2k_df=pd.DataFrame(Above2k)
Above2k_df=Above2k_df.reset_index()
Above2k_df.columns =['Item Description', 'Count']
Above2k_df=Above2k_df[Above2k_df['Count'] >= 500 ]
Above2k_df.drop(['Count'], axis = 1, inplace = True)

In [224]:
Above2k_df

,Item Description
0,FASTENERS/SCREWS/NAILS/BOLTS ETC
1,KEY KWIKSET KW1-ACE250PK
2,BIRDSEED WILDBIRD 20#ACE
3,KEY SCHLAGE SC1-ACE250PK
4,CONTRACTOR BAGS 3MIL. 20CNT
5,TRAP SPIDER & CRICKET PK
6,PEAK WASH/DEICER -25
7,COLOR SAMPLE BM BASE1 PT
8,SCREEN REPAIR
9,TOP SOIL 40#


In [225]:
#basket_cleaned_fine=basket_cleaned_df
basket_cleaned_df=basket_clean[basket_clean['Item Description'].isin(Above2k_df['Item Description'])]

### Removing receipts having values less than 2

In [226]:
basket_cleaned_df['Net Sales Units']= pd.to_numeric(basket_cleaned_df['Net Sales Units'], errors = 'coerce')

s = basket_cleaned_df.groupby(['Receipt Number'])['Item Description'].count()

In [250]:

itemsInR = pd.DataFrame(s)

In [251]:
# itemsInR = itemsInR.reset_index()

In [271]:
itemsInR 

,Item Description
Receipt Number,
015338,1
015339,1
015344,1
015352,1
015354,3
...,...
J00117,2
J00119,1
X21252,2


In [253]:
itemsInR.to_csv("/Users/ashvtol/Documents/dsfProjectData/transPerRecBethpage.csv")

In [270]:
itemsInR['Item Description']

Receipt Number
015338    1
015339    1
015344    1
015352    1
015354    3
         ..
J00117    2
J00119    1
X21252    2
X21260    2
X21261    2
Name: Item Description, Length: 212575, dtype: int64

In [230]:
itemsInR = itemsInR[itemsInR['Item Description']>=2]

In [231]:
itemsInR.shape

(56173, 2)

In [232]:
itemsInR.columns = ['Receipt Number', 'Item in receipt']

In [233]:
savedR = itemsInR

In [234]:
savedR

,Receipt Number,Item in receipt
4,015354,3
5,015355,2
6,015358,2
15,015397,4
29,015447,2
...,...,...
212568,J00109,2
212570,J00117,2
212572,X21252,2
212573,X21260,2


In [235]:
fileteredReceipt = savedR.merge(basket_cleaned_df, on='Receipt Number', how='left')

In [236]:
basket_cleaned_fine = fileteredReceipt

### Apriori

In [237]:
#Grouping the items bought together receipt number wise
basket_cleaned_fine['Net Sales Units']= pd.to_numeric(basket_cleaned_fine['Net Sales Units'], errors = 'coerce')
basket=basket_cleaned_fine.groupby(['Receipt Number', 'Item Description'])['Net Sales Units'].sum().unstack().reset_index().fillna(0).set_index('Receipt Number')

In [238]:
#Encoding the quantities purchased together 

def encode_units(x):
    if int(x) <= 0:
        return int(0)
    if int(x) >= 1:
        return int(1)

# def encode_units(x):
#     if x <= 0:
#         return 0
#     if x >= 1:
#         return 1

basket_sets = basket.applymap(encode_units)
#basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [239]:
# basket_sets['FASTENERS'].value_counts()
basket_sets

Item Description,39PAR20/HAL/XEN/NFL/120V,40W A15 APPLIANCE CLR,40W A15 APPLIANCE FR,50LB CALCIUM CHLORIDE PELLETS,65BR/FL/RP 120V,65BR30 REFLECTOR 2/PK,"9"" PLASTIC TRAY LINER",9A19 LED 120V 2700K 4PK,A/C COVER 18X27X16,ACE 33GAL DRWST BAG 20CT,ACE FERT 4STEP 5M,ACE PREM ICE MELTER JUG,ACE TOP SOIL 40#,AJAX LQD 2X ULTRA 50 OZ,ANGLE BROOM,ANNIE B'S CARAMEL ASSORTED,ANTIFRZ ARTIC RV 1GL,BACK TO CAMPUS 25% OFF,BAG MATTRESS 70X92 QN/FL,BAIT ANT MAXATTRAX 4CT,BAIT ANT RAID 4CT,BATTERY AA 16PK ENERGIZR,BATTERY AA 8PK ENERGIZER,BATTERY ALK AA 16WIDE,BATTERY ALKLINE 12 VOLT,BATTERY WATCH/CALC 2032,BATTRY ALKLN DURA AA CD4,BATTRY ALKLN DURA AA CD8,BATTRY LITHUM 3V 2032PK2,BEN INT EGGSHL BASE1 GL,BEN INT FLAT BASE1 GL,BEN INT S-G WHITE GL,BEN INT S-G WHITE QT,BERRY BLAST SUET,BIRD BLEND SUET,BIRDSEED WILDBIRD 20#ACE,BLADE RAZOR #9 1EDGE10PK,BLEACH LIQ REG CONC 64OZ,BRUSH/ICESCRAPRTELESCP42,BULB FLUOR F40T12 CW 2PK,BULB HAL A19 43W SW 4PK,BULB SOFTWHT 43W ACE 4PK,BUNDLED FIREWOOD,CANOPY 10' X 10',CEILING WB U-FLT WHT GL,CITRONELLA TORCH FUEL-64OZ,"CLAMP 1/2 TO 1-1/4""SS",CLASSIC 20OZ,CLEANR DRN DRNMX GEL32OZ,CLEANR GLAS19OZ SPRAYWAY,CLICK N FLAME LIGHTERS,CMN MILK CHOC BAR,CMN MILKCHOC ALMD BAR,COLOR SAMPLE BM BASE1 PT,COLOR SAMPLE BM BASE2 PT,COLOR SAMPLE BM BASE3 PT,CONTRACTOR BAGS 3MIL. 20CNT,CONTRACTOR BAGS 50CT 3MIL,COSTELLO'S CONTRACTOR MIX 3LB,COSTELLO'S GIFT CARD,COSTELLO'S MERCHANDISE CARD,COSTELLO'S SUN & SHADE 3#,CRABGRSS PRVNTR STEP1 5M,CUTTER BUG FREE 32 OZ,DECORA WALLPLATE 1G WHT,DECORA WALLPLATE 2G WHT,DEEPWELL PAINT TRY LINER,DIET COKE 20OZ,DR PEPPER 20OZ,DRAIN OPNR HD LIQ 32OZ,DRAIN OPNR HD LIQ 64OZ,DRANO MAX GEL CLOG REMOV,DROPCLOTH 9' X 12' 1MIL,DROPCLOTH 9'X12' .5MIL,DROPCLOTH 9X12 .7MIL 3PK,DRUM BAGS 55GAL 2MIL. 20CNT,"DUCT TAPE 1.88""X60YD ACE",DUCT TAPE 60YDS GRAY ACE,EZ SEED SUN&SHADE 10#,EZ SEED SUN&SHADE 3.75#,FASTENERS/SCREWS/NAILS/BOLTS ETC,FERTILIZR 4STEP PRGM 5M,"FILTR A/C POLY15X24X1/4""",FLEX SEAL 14 OZ CLEAR,"FLEX TAPE BLACK 4""X5'","FLEX TAPE WHITE 4""X5'",FOAM BRUSH 10PC WOODHNDL,FOGGER YARD CUTR 16OZ2PK,FUZE ICE TEA,GORILLA GLUE ORGNL 2OZ,GRASS SEED SUN&SHADE 3#,GREAT STUFF BIG GAP 12OZ,GROOVY KEY,GRUB KILLER+24HR10#GRAN,HAIR SNAKE DRAIN OPENER,HTH SHOCK TREAT,ICE MELT 20# BAG ACE,JNT CMPND ALL PURP 1.75P,JNT CMPND ALL PURP 3.5QT,KEY ARROW AR1-ACE,KEY KWIKSET KW1,KEY KWIKSET KW1-ACE250PK,KEY MASTER M1-ACE250PK,KEY SCHLAGE SC1-ACE250PK,KEY SCHLAGE SC1250PK,KNIFE UTIL RETRACT QUICK,LAWN FOOD 5M,LED FEIT A19 60W EQ WW,LIBERTY GARDEN SEEDS-ASSORTED,"LIGHTS MINI 100CT 2.5"" CLEAR",LIQUID ANT BAIT 6PK,MDA DONATION $1,MONSTR 6OUTLET STRIP WHT,MR CLEAN ERASER 2PK,MR. CLEAN EXTRA PWR ERSR,MSKG TAPE ORIGNL .94x60,MSKG TAPE ORIGNL 1.41x60,MSKG TAPE ORIGNL 1.88x60,MULCH HAMPTONS DYED BLACK 2CU,MURESCO CEILING WHITE GAL,NESTLE PURE LIFE 24PK.5L,NEW MEMBER 1 $5 OFF,OIL 2 CYCLE 2.6 OZ ACE,OIL SUNFLOWER BSEED 20LB,OMNI ALL IN ONE CLIP BOX,ORANGE BURST SUET,PAINT TRAY LINER,PAINTBRS CHIP 1.5WT BRSL,PAINTBRS CHIP 2.5WT BRSL,"PAINTBRSH CHIP 1""WH BRSL","PAINTBRSH CHIP 2""WT BRSL","PAINTBRSH CHIP 3""WT BRSL","PAINTBRSH FOAM 1"" JEN","PAINTBRSH FOAM 2"" JEN",PAINTBRSH FOAM3PK AST EP,PEAK BUG WASH +32,PEAK WASH/DEICER -25,PEANUT CRUNCH SUET,PEQUA H/D DRAIN OPENER QT,PEQUA H/D DRAIN OPNR 1/2G,PLASTIC DEEP WELL TRAY,PLUMBERS PUTTY 14OZ,PNT-FRCE 9X12 1MIL PLASTIC DROP,POLAR FLEECE GLOVES,POT SOIL MOIST CONT 2 CF,POTTING MIX W/FERT 8QT,PROPANE FUEL 16OZ 2PK,"PUTTY KNIFE 1.5""PLAS ACE",RECEPTACLE WALLPLATE1GWH,RED BULL ENRGY DRNK8.4OZ,RED CROSS DONATION,REG SEL INT EGGSHL B1 GL,REG SEL INT EGGSHL B2 GL,REG SEL INT MATTE BS1 GL,ROLLR WHT DOVE 9X3/8 3PK,ROUNDUP RTU 30OZ BONUS,"SCRAPER/SNOWBRUSH 22""",SCRAPR RAZOR5BLADES ACE,SCREEN REPAIR,"SHOVEL SNOW 18""BLADE","SHOVEL SNOW D-HNDL 36""",SILICONE II K&B CLR2.8OZ,SJ 25LBS CC PELLETS,SJ 50LBS CC PELLETS,SOAP LIFT OVAL-ASST COLORS,SPACKL VINYL 1/2PT ACE,SPIDER TRAP 4PK.,SPRITE 20OZ,SUET CAKE HI ENERGY 8PK,SUET NUTTY BUTTR BIRDWATCHR BEST,TAPE ELECT3/4X60'VYL ACE,THANK YOU FOR SHOPPIN

### Sniff Test

In [285]:
b = pd.DataFrame(basket_sets)
b.shape

(56173, 197)

In [296]:
a = b[ (b['WEED & FEED 5M'] == 1)]

In [297]:
a.shape

(822, 197)

#### Applying apriori algorithm on the grouped items

In [240]:

frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [241]:
frequent_itemsets

,support,itemsets
0,0.012675,(BEN INT EGGSHL BASE1 GL)
1,0.010806,(BERRY BLAST SUET)
2,0.012462,(BIRD BLEND SUET)
3,0.050718,(BIRDSEED WILDBIRD 20#ACE)
4,0.011821,(CEILING WB U-FLT WHT GL)
5,0.015488,(CLEANR GLAS19OZ SPRAYWAY)
6,0.026027,(COLOR SAMPLE BM BASE1 PT)
7,0.013814,(COLOR SAMPLE BM BASE2 PT)
8,0.027344,(CONTRACTOR BAGS 3MIL. 20CNT)
9,0.475887,(FASTENERS/SCREWS/NAILS/BOLTS ETC)


In [242]:
#Applying association rules to obtain the antecedants and the precedants
rules = association_rules(frequent_itemsets, metric="lift", min_threshold = 0.01)
rules[(rules['lift'] >= 1) & (rules['confidence'] >= 0.1)]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(KEY KWIKSET KW1-ACE250PK),(KEY SCHLAGE SC1-ACE250PK),0.052534,0.046161,0.019867,0.378177,8.192569,0.017442,1.533940
5,(KEY SCHLAGE SC1-ACE250PK),(KEY KWIKSET KW1-ACE250PK),0.046161,0.052534,0.019867,0.430390,8.192569,0.017442,1.663357
6,(LAWN FOOD 5M),(WEED & FEED 5M),0.015595,0.014633,0.012070,0.773973,52.890953,0.011842,4.359501
7,(WEED & FEED 5M),(LAWN FOOD 5M),0.014633,0.015595,0.012070,0.824818,52.890953,0.011842,5.619314


In [243]:
rules.sort_values(by=['lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(LAWN FOOD 5M),(WEED & FEED 5M),0.015595,0.014633,0.012070,0.773973,52.890953,0.011842,4.359501
7,(WEED & FEED 5M),(LAWN FOOD 5M),0.014633,0.015595,0.012070,0.824818,52.890953,0.011842,5.619314
4,(KEY KWIKSET KW1-ACE250PK),(KEY SCHLAGE SC1-ACE250PK),0.052534,0.046161,0.019867,0.378177,8.192569,0.017442,1.533940
5,(KEY SCHLAGE SC1-ACE250PK),(KEY KWIKSET KW1-ACE250PK),0.046161,0.052534,0.019867,0.430390,8.192569,0.017442,1.663357
0,(KEY KWIKSET KW1-ACE250PK),(FASTENERS/SCREWS/NAILS/BOLTS ETC),0.052534,0.475887,0.014402,0.274144,0.576070,-0.010598,0.722062
1,(FASTENERS/SCREWS/NAILS/BOLTS ETC),(KEY KWIKSET KW1-ACE250PK),0.475887,0.052534,0.014402,0.030263,0.576070,-0.010598,0.977034
2,(KEY SCHLAGE SC1-ACE250PK),(FASTENERS/SCREWS/NAILS/BOLTS ETC),0.046161,0.475887,0.011233,0.243347,0.511356,-0.010734,0.692673
3,(FASTENERS/SCREWS/NAILS/BOLTS ETC),(KEY SCHLAGE SC1-ACE250PK),0.475887,0.046161,0.011233,0.023605,0.511356,-0.010734,0.976898


### Merging Weather Data

In [343]:
weather =  pd.read_csv('/Users/ashvtol/Documents/dsfProjectData/weatherData.csv')

In [341]:
# weather['Date']= weather['Date'].astype(str) 

In [344]:
weather

,Date,Temp,PCP24,sd
0,1/1/2015,28.240000,0.000000,0.00
1,1/1/2016,39.160000,0.008800,0.00
2,1/1/2017,41.480000,0.000417,0.00
3,1/1/2018,10.640000,0.000000,0.16
4,1/1/2019,50.097561,0.067561,0.00
...,...,...,...,...
1783,9/9/2015,76.125000,0.000000,0.00
1784,9/9/2016,76.888889,0.000000,0.00
1785,9/9/2017,59.400000,0.000000,0.00
1786,9/9/2018,59.240000,0.003478,0.00


In [327]:
data

,Unnamed: 0,Date,Transaction Time,Customer Number,Receipt Number,Store #,Store Name,Scanned UPC,Item Number,Item Description,Net Sales Units,Net Sales,Cost,Gross Margin,Gross Margin %,Department Code,Department Name,Class Code,Class Name,Fineline Code,Fineline Name,Item was Scanned,MIP Promo ID,Promo/Discount,Dynamic Promo ID,Actual Price,Retail Price,Actual-Retail,Taxable,Tender Type,$ Off Retail,Zip Code,Zip Plus-4,Loyalty ID,Clerk,Item Type,Line #,Line Item Transaction Type,Pricing Source,Return Code
0,0,4/1/2017,6:52:42,*5,G13674,9,11730 BETHPAGE,OTHER,56,FASTENERS,1,0.05,0.02,0.03,60.00%,56,FASTENERS,HIL,"HILLMAN GROUP, INC.",NaN,BLANK,5,NaN,NaN,0,0.05,0,$0.05,Y,Cash Tender,-0.05,NaN,NaN,NaN,JOHN FILIPE,Sale,1,Sale,Manual Override,NaN
1,1,4/1/2017,7:02:55,290074,D70801,O,15863 COPIAGUE,OTHER,7158314,ACE FERT 4STEP 5M,1,64.99,43.5,21.49,33.07%,70,"LAWN, GARDEN & FARM SUPPLIES",744,FERTILIZERS,74401,LAWN (DRY BAG),5,1042017,S,1871,64.99,67.99,($3.00),Y,Bankcard Tender,3,11717,8017,1.92617e+09,LISA JUDGE,Sale,1,Sale,Promotion,NaN
2,2,4/1/2017,7:02:55,290074,D70801,O,15863 COPIAGUE,OTHER,IS117235,7158314 $20 INST SAVINGS,-1,-20,0,-20,100.00%,I1,ACE REWARDS INSTANT SAVINGS,IS1,ACE COUPON/CREDIT ITEMS,IS106,ACE COUPON/CREDIT ITEMS,5,170451AC,S,1871,20,20,$0.00,N,Bankcard Tender,0,11717,8017,1.92617e+09,LISA JUDGE,Return,2,Return,Promotion,NaN
3,3,4/1/2017,7:14:22,191376,A39683,F,14822 NORTHPORT,82901195757,78832,"ACE DECK NAIL16D3.5""ST1#",1,7.99,1.89,6.1,76.35%,56,FASTENERS,574,PACKAGE NAILS,57403,1 & 5LB NAILS AND STAPLES,1,NaN,NaN,0,7.99,7.99,$0.00,Y,Cash Tender,0,11768,760,1.91436e+09,GARY GASPAR LIPARI,Sale,1,Sale,Retail,NaN
4,4,4/1/2017,7:14:22,191376,A39683,F,14822 NORTHPORT,AUTO,9269862,CMN Donations,1,0.32,0.16,0.16,50.00%,DO,DONTATIONS,DON,DONATIONS,NaN,BLANK,4,NaN,*,0,0.32,0,$0.32,N,Cash Tender,-0.32,11768,760,1.91436e+09,GARY GASPAR LIPARI,Sale,2,Sale,Manual Override,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32955538,15627494,9/30/2016,21:14:44,500868,I38056,3,3164 NESCONSET,51643061880,8009730,"CORD STRETCH 48"" HVY DTY",1,2.1,1.68,0.42,20.00%,80,SPORTING GOODS,856,STRETCH CORDS/TARP STRAPS,85601,STRETCH CORDS/HOOKS,1,NaN,NaN,0,2.1,5.99,($3.89),Y,Alternate Tender,3.89,11787,NaN,1.98164e+09,ANDRIY CHEVEVAN,Sale,1,Sale,Category Price Plan,NaN
32955539,15627495,9/30/2016,21:14:44,500868,I38056,3,3164 NESCONSET,82901731900,73190,TARP POLY BLUE/BRN 6X8,1,3.06,2.45,0.61,19.93%,70,"LAWN, GARDEN & FARM SUPPLIES",774,TARPS,77401,POLY TARPS,1,NaN,NaN,0,3.06,7.99,($4.93),Y,Alternate Tender,4.93,11787,NaN,1.98164e+09,ANDRIY CHEVEVAN,Sale,2,Sale,Category Price Plan,NaN
32955540,15627496,9/30/2016,21:14:44,500868,I38056,3,3164 NESCONSET,42854024986,1365527,SPRAYER ORBITAL 24 OZ,1,2,1.6,0.4,20.00%,10,CLEANING SUPPLIES,105,GLASS/PLASTIC CLEANERS,10505,SPRAYERS,1,NaN,NaN,0,2,3.99,($1.99),Y,Alternate Tender,1.99,11787,NaN,1.98164e+09,ANDRIY CHEVEVAN,Sale,3,Sale,Category Price Plan,NaN
32955541,15627497,9/30/2016,21:29:00,507179,B84423,S,16147 CALDWELL,49000007640,CO5964,SPRITE 20OZ,1,1.19,0.95,0.24,20.17%,9,SODA/ETC,927,CANDY AND FOODS,92706,BEVERAGES,1,NaN,NaN,0,1.19,1.99,($0.80),Y,Alternate Tender,0.8,7063,NaN,1.9818e+09,GEORGE TAYLOR,Sale,1,Sale,Category Price Plan,NaN


In [347]:
data

,Date,Transaction Time,Customer Number,Receipt Number,Store #,Store Name,Scanned UPC,Item Number,Item Description,Net Sales Units,Net Sales,Cost,Gross Margin,Gross Margin %,Department Code,Department Name,Class Code,Class Name,Fineline Code,Fineline Name,Item was Scanned,MIP Promo ID,Promo/Discount,Dynamic Promo ID,Actual Price,Retail Price,Actual-Retail,Taxable,Tender Type,$ Off Retail,Zip Code,Zip Plus-4,Loyalty ID,Clerk,Item Type,Line #,Line Item Transaction Type,Pricing Source,Return Code
0,4/1/2017,6:52:42,*5,G13674,9,11730 BETHPAGE,OTHER,56,FASTENERS,1,0.05,0.02,0.03,60.00%,56,FASTENERS,HIL,"HILLMAN GROUP, INC.",NaN,BLANK,5,NaN,NaN,0,0.05,0,$0.05,Y,Cash Tender,-0.05,NaN,NaN,NaN,JOHN FILIPE,Sale,1,Sale,Manual Override,NaN
1,4/1/2017,7:02:55,290074,D70801,O,15863 COPIAGUE,OTHER,7158314,ACE FERT 4STEP 5M,1,64.99,43.5,21.49,33.07%,70,"LAWN, GARDEN & FARM SUPPLIES",744,FERTILIZERS,74401,LAWN (DRY BAG),5,1042017,S,1871,64.99,67.99,($3.00),Y,Bankcard Tender,3,11717,8017,1.92617e+09,LISA JUDGE,Sale,1,Sale,Promotion,NaN
2,4/1/2017,7:02:55,290074,D70801,O,15863 COPIAGUE,OTHER,IS117235,7158314 $20 INST SAVINGS,-1,-20,0,-20,100.00%,I1,ACE REWARDS INSTANT SAVINGS,IS1,ACE COUPON/CREDIT ITEMS,IS106,ACE COUPON/CREDIT ITEMS,5,170451AC,S,1871,20,20,$0.00,N,Bankcard Tender,0,11717,8017,1.92617e+09,LISA JUDGE,Return,2,Return,Promotion,NaN
3,4/1/2017,7:14:22,191376,A39683,F,14822 NORTHPORT,82901195757,78832,"ACE DECK NAIL16D3.5""ST1#",1,7.99,1.89,6.1,76.35%,56,FASTENERS,574,PACKAGE NAILS,57403,1 & 5LB NAILS AND STAPLES,1,NaN,NaN,0,7.99,7.99,$0.00,Y,Cash Tender,0,11768,760,1.91436e+09,GARY GASPAR LIPARI,Sale,1,Sale,Retail,NaN
4,4/1/2017,7:14:22,191376,A39683,F,14822 NORTHPORT,AUTO,9269862,CMN Donations,1,0.32,0.16,0.16,50.00%,DO,DONTATIONS,DON,DONATIONS,NaN,BLANK,4,NaN,*,0,0.32,0,$0.32,N,Cash Tender,-0.32,11768,760,1.91436e+09,GARY GASPAR LIPARI,Sale,2,Sale,Manual Override,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32955538,9/30/2016,21:14:44,500868,I38056,3,3164 NESCONSET,51643061880,8009730,"CORD STRETCH 48"" HVY DTY",1,2.1,1.68,0.42,20.00%,80,SPORTING GOODS,856,STRETCH CORDS/TARP STRAPS,85601,STRETCH CORDS/HOOKS,1,NaN,NaN,0,2.1,5.99,($3.89),Y,Alternate Tender,3.89,11787,NaN,1.98164e+09,ANDRIY CHEVEVAN,Sale,1,Sale,Category Price Plan,NaN
32955539,9/30/2016,21:14:44,500868,I38056,3,3164 NESCONSET,82901731900,73190,TARP POLY BLUE/BRN 6X8,1,3.06,2.45,0.61,19.93%,70,"LAWN, GARDEN & FARM SUPPLIES",774,TARPS,77401,POLY TARPS,1,NaN,NaN,0,3.06,7.99,($4.93),Y,Alternate Tender,4.93,11787,NaN,1.98164e+09,ANDRIY CHEVEVAN,Sale,2,Sale,Category Price Plan,NaN
32955540,9/30/2016,21:14:44,500868,I38056,3,3164 NESCONSET,42854024986,1365527,SPRAYER ORBITAL 24 OZ,1,2,1.6,0.4,20.00%,10,CLEANING SUPPLIES,105,GLASS/PLASTIC CLEANERS,10505,SPRAYERS,1,NaN,NaN,0,2,3.99,($1.99),Y,Alternate Tender,1.99,11787,NaN,1.98164e+09,ANDRIY CHEVEVAN,Sale,3,Sale,Category Price Plan,NaN
32955541,9/30/2016,21:29:00,507179,B84423,S,16147 CALDWELL,49000007640,CO5964,SPRITE 20OZ,1,1.19,0.95,0.24,20.17%,9,SODA/ETC,927,CANDY AND FOODS,92706,BEVERAGES,1,NaN,NaN,0,1.19,1.99,($0.80),Y,Alternate Tender,0.8,7063,NaN,1.9818e+09,GEORGE TAYLOR,Sale,1,Sale,Category Price Plan,NaN


In [348]:
d2 = data.merge(weather, on='Date', how='left')

In [353]:
d2.head(100)

,Date,Transaction Time,Customer Number,Receipt Number,Store #,Store Name,Scanned UPC,Item Number,Item Description,Net Sales Units,Net Sales,Cost,Gross Margin,Gross Margin %,Department Code,Department Name,Class Code,Class Name,Fineline Code,Fineline Name,Item was Scanned,MIP Promo ID,Promo/Discount,Dynamic Promo ID,Actual Price,Retail Price,Actual-Retail,Taxable,Tender Type,$ Off Retail,Zip Code,Zip Plus-4,Loyalty ID,Clerk,Item Type,Line #,Line Item Transaction Type,Pricing Source,Return Code,Temp,PCP24,sd
0,4/1/2017,6:52:42,*5,G13674,9,11730 BETHPAGE,OTHER,56,FASTENERS,1,0.05,0.02,0.03,60.00%,56,FASTENERS,HIL,"HILLMAN GROUP, INC.",NaN,BLANK,5,NaN,NaN,0,0.05,0,$0.05,Y,Cash Tender,-0.05,NaN,NaN,NaN,JOHN FILIPE,Sale,1,Sale,Manual Override,NaN,38.540541,0.074324,0.0
1,4/1/2017,7:02:55,290074,D70801,O,15863 COPIAGUE,OTHER,7158314,ACE FERT 4STEP 5M,1,64.99,43.5,21.49,33.07%,70,"LAWN, GARDEN & FARM SUPPLIES",744,FERTILIZERS,74401,LAWN (DRY BAG),5,1042017,S,1871,64.99,67.99,($3.00),Y,Bankcard Tender,3,11717,8017,1.92617e+09,LISA JUDGE,Sale,1,Sale,Promotion,NaN,38.540541,0.074324,0.0
2,4/1/2017,7:02:55,290074,D70801,O,15863 COPIAGUE,OTHER,IS117235,7158314 $20 INST SAVINGS,-1,-20,0,-20,100.00%,I1,ACE REWARDS INSTANT SAVINGS,IS1,ACE COUPON/CREDIT ITEMS,IS106,ACE COUPON/CREDIT ITEMS,5,170451AC,S,1871,20,20,$0.00,N,Bankcard Tender,0,11717,8017,1.92617e+09,LISA JUDGE,Return,2,Return,Promotion,NaN,38.540541,0.074324,0.0
3,4/1/2017,7:14:22,191376,A39683,F,14822 NORTHPORT,82901195757,78832,"ACE DECK NAIL16D3.5""ST1#",1,7.99,1.89,6.1,76.35%,56,FASTENERS,574,PACKAGE NAILS,57403,1 & 5LB NAILS AND STAPLES,1,NaN,NaN,0,7.99,7.99,$0.00,Y,Cash Tender,0,11768,760,1.91436e+09,GARY GASPAR LIPARI,Sale,1,Sale,Retail,NaN,38.540541,0.074324,0.0
4,4/1/2017,7:14:22,191376,A39683,F,14822 NORTHPORT,AUTO,9269862,CMN Donations,1,0.32,0.16,0.16,50.00%,DO,DONTATIONS,DON,DONATIONS,NaN,BLANK,4,NaN,*,0,0.32,0,$0.32,N,Cash Tender,-0.32,11768,760,1.91436e+09,GARY GASPAR LIPARI,Sale,2,Sale,Manual Override,NaN,38.540541,0.074324,0.0
5,4/1/2017,7:16:09,99996,2815,S,16147 CALDWELL,OTHER,10529,SCRAPR RAZOR5BLADES ACE,1,1.37,1.42,-0.05,-3.65%,14,PAINT & PAINT SUPPLIES,135,KNIVES/SCRAPERS,13504,RAZOR SCRAPERS,5,NaN,NaN,0,1.37,5.49,($4.12),N,Charge Tender,4.12,NaN,NaN,NaN,GREGORY BOSSET,Sale,1,Sale,Category Price Plan,NaN,38.540541,0.074324,0.0
6,4/1/2017,7:16:09,99996,2815,S,16147 CALDWELL,OTHER,4547758,LAUNDRY FAUCET 2H CH OB,1,12.25,12.69,-0.44,-3.59%,40,PLUMBING SUPPLIES & FIXTURES,400,FAUCETS,40004,LAUNDRY AND BAR,5,NaN,NaN,0,12.25,29.99,($17.74),N,Charge Tender,17.74,NaN,NaN,NaN,GREGORY BOSSET,Sale,2,Sale,Category Price Plan,NaN,38.540541,0.074324,0.0
7,4/1/2017,7:16:09,99996,2815,S,16147 CALDWELL,OTHER,40924,LAUNDRY TUB 33X23X25 1PK,1,29.81,27.55,2.26,7.58%,40,PLUMBING SUPPLIES & FIXTURES,447,LAUNDRY TUBS,44701,LAUNDRY TUBS,5,NaN,NaN,0,29.81,34.99,($5.18),N,Charge Tender,5.18,NaN,NaN,NaN,GREGORY BOSSET,Sale,3,Sale,Category Price Plan,NaN,38.540541,0.074324,0.0
8,4/1/2017,7:20:14,*5,E37837,L,15348 ROCKY POINT,76174334258,24063,"RULE TAPE 1""X25'POWRLOCK",1,12.99,9.26,3.73,28.71%,20,HAND TOOLS & TOOL ACCESSORIES,232,MEASURING TOOLS/REFILLS,23201,TAPE RULES,1,NaN,NaN,0,12.99,12.99,$0.00,Y,Cash Tender,0,NaN,NaN,NaN,THOMAS KRAJEWSKI,Sale,1,Sale,Retail,NaN,38.540541,0.074324,0.0
9,4/1/2017,7:21:00,500851,D70802,O,15863 COPIAGUE,41911000505,1005412,CLEANR GLAS19OZ SPRAYWAY,2,3.46,2.89,0.57,16.47%,10,CLEANING SUPPLIES,105,GLASS/PLASTIC CLEANERS,10504,GLASS CLEANER (AEROSOL),1,NaN,NaN,0,1.73,2.99,($1.26),Y,Cash Tender,1.26,11762,NaN,1.98164e+09,LISA JUDGE,Sale,1,Sale,Category Price Plan,NaN,38.540541,0.074324,0.0


In [352]:
# d2.to_csv('/Users/ashvtol/Documents/dsfProjectData/fullDatawithWeather.csv')